<a href="https://colab.research.google.com/github/trifarameliniaa/SKRIPSI/blob/main/Skripsi_Citra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
from PIL import Image
import PIL.ImageOps

In [24]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [35]:
img = cv2.imread('/content/penyakit-daun-tanaman.jpg')

In [26]:
img.size

150975

In [36]:
img_array = np.array(img)
img_array

array([[[14, 71, 40],
        [13, 70, 39],
        [15, 69, 39],
        ...,
        [ 6, 44, 26],
        [ 6, 44, 26],
        [ 6, 44, 26]],

       [[14, 71, 40],
        [14, 71, 40],
        [15, 69, 39],
        ...,
        [ 5, 44, 23],
        [ 5, 43, 25],
        [ 5, 43, 25]],

       [[15, 72, 41],
        [14, 71, 40],
        [14, 71, 40],
        ...,
        [ 5, 44, 23],
        [ 5, 43, 25],
        [ 5, 43, 25]],

       ...,

       [[ 1, 56, 11],
        [ 0, 56, 11],
        [ 0, 55, 10],
        ...,
        [24, 62, 44],
        [24, 62, 44],
        [24, 61, 45]],

       [[ 0, 56, 11],
        [ 0, 56, 11],
        [ 0, 57, 12],
        ...,
        [22, 59, 43],
        [22, 59, 43],
        [22, 58, 44]],

       [[ 0, 58, 13],
        [ 0, 58, 13],
        [ 0, 58, 13],
        ...,
        [21, 57, 43],
        [22, 57, 43],
        [22, 56, 45]]], dtype=uint8)